# Trening klasyfikatora

In [1]:
from supernova.dataset import get_dataset_split, get_data_loaders
from supernova.modeling.model import SupernovaClassifierV1Config, SupernovaClassifierV1
from supernova.modeling.train import SupernovaTraining, get_trainer

In [2]:
datasets = get_dataset_split(
    "../data/processed/training_set.pkl",
    val_split=0.2,
    test_split=0.1,
)
loaders = get_data_loaders(datasets, batch_size=32, num_workers=4)

In [3]:
config = SupernovaClassifierV1Config(
    metadata_input_size=10,
    metadata_num_hidden_layers=2,
    metadata_hidden_size=32,
    metadata_output_size=16,
    lightcurve_input_size=4,
    lightcurve_num_hidden_layers=2,
    lightcurve_hidden_size=32,
    classifier_hidden_size=64,
    classifier_num_hidden_layers=2,
    num_classes=14,
    dropout=0.2,
)
model = SupernovaClassifierV1(config)

In [4]:
training = SupernovaTraining(model, learning_rate=1e-3)
trainer = get_trainer(
    epochs=3,
    checkpoint_dir="../models/checkpoints",
    early_stop_patience=10,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/home/mgarbowski/repos/fo-projekt/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [5]:
trainer.fit(training, loaders["train"], loaders["val"])

/home/mgarbowski/repos/fo-projekt/.venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:881: Checkpoint directory /home/mgarbowski/repos/fo-projekt/models/checkpoints exists and is not empty.

  | Name      | Type                  | Params | Mode  | FLOPs
--------------------------------------------------------------------
0 | model     | SupernovaClassifierV1 | 105 K  | train | 0    
1 | criterion | CrossEntropyLoss      | 0      | train | 0    
--------------------------------------------------------------------
105 K     Trainable params
0         Non-trainable params
105 K     Total params
0.422     Total estimated model params size (MB)
37        Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Trening się kończy bo przez braki w danych `loss=nan`